In [39]:
import os
import pickle
from glob import iglob
import numpy as np
import librosa
import random

DATA_AUDIO_DIR = './test'
OUTPUT_DIR = './output'
TEST_DIR = './Voices_mult/test'
TRAIN_DIR = './Voices_mult/train'
OUTPUT_TRAIN = os.path.join(OUTPUT_DIR, 'train')
OUTPUT_TEST = os.path.join(OUTPUT_DIR, 'test')
AUDIO_LENGTH = 10000
FINAL_SR = 8000
manipulation_const = 0.002

class_ids = {
    'one': 0,
    'two': 1,
    'three': 2,
    'four': 3,
    'five': 4,
}
def extract_id(wav_filename):
    if '1' in wav_filename:
        return class_ids.get('one')
    elif '2' in wav_filename:
        return class_ids.get('two')
    elif '3' in wav_filename:
        return class_ids.get('three')
    elif '4' in wav_filename:
        return class_ids.get('four')
    elif '5' in wav_filename:
        return class_ids.get('five')
    else:
        return class_ids.get('unlabelled')

def read_audio_from_filename(filename, final_sr):
    audio, _ = librosa.load(filename, sr=final_sr, mono=True)
    audio = audio.reshape(-1, 1)

In [34]:

def manipulate(data):
        noise = np.random.randn(len(data))
        augmented_data = data + manipulation_const * noise
        augmented_data = augmented_data.astype(type(data[0]))
        return augmented_data

for i, wav_filename in enumerate(iglob(os.path.join('DATA_AUDIO_DIR', '**/**.wav'), recursive=True)):                 
        print(wav_filename)
        class_id = extract_id(wav_filename)
        file_name = os.path.splitext(os.path.basename(wav_filename))[0]
        dest_path = os.path.dirname(wav_filename)
        
        output_folder = './pkls'
        
        audio, sr = librosa.load(wav_filename, mono=True)
        
        for j in range(len(audios)): 
            output_filename = os.path.join(output_folder, file_name + str(j) + '.wav')
            librosa.output.write_wav(output_filename, audio, sr)


In [5]:
DATA_AUDIO_DIR = './test'
OUTPUT_DIR = './pkls'
import random


def extract_id(wav_filename):
    if '1' in wav_filename:
        return class_ids.get('one')
    elif '2' in wav_filename:
        return class_ids.get('two')
    elif '3' in wav_filename:
        return class_ids.get('three')
    elif '4' in wav_filename:
        return class_ids.get('four')
    elif '5' in wav_filename:
        return class_ids.get('five')
    else:
        return class_ids.get('unlabelled')

def read_audio_from_filename(filename, final_sr):
    audio, _ = librosa.load(filename, sr=final_sr, mono=True)
    audio = audio.reshape(-1, 1)
    return audio

def prepare_all_data():
    for i, wav_filename in enumerate(iglob(os.path.join(DATA_AUDIO_DIR, '**/**.wav'), recursive=True)):
        class_id = extract_id(wav_filename)
        audio_b = read_audio_from_filename(wav_filename, final_sr=FINAL_SR)
        audio_buf_final = (audio_b - np.mean(audio_b)) / np.std(audio_b)
        original_length = len(audio_buf_final)
        print(i, wav_filename, original_length, np.round(np.mean(audio_buf_final), 4), np.std(audio_buf_final))
        
        if original_length < AUDIO_LENGTH:
            audio_buf_final = np.concatenate((audio_buf_final, np.zeros(shape=(AUDIO_LENGTH - original_length, 1))))
            print('PAD New length =', len(audio_buf_final))
        elif original_length > AUDIO_LENGTH:
            audio_buf_final = audio_buf_final[0:AUDIO_LENGTH]
            print('CUT New length =', len(audio_buf_final))

        output_folder = OUTPUT_DIR

        output_filename = os.path.join(output_folder, str(i) + '.pkl')
        file_name = os.path.splitext(os.path.basename(wav_filename))[0]
        pkl_out = {'class_id': class_id,  'audio': audio_buf_final, 'sr': FINAL_SR}
        with open(output_filename, 'wb') as w:
            pickle.dump(pkl_out, w)


if __name__ == '__main__':
    prepare_all_data()

0 ./test\1 achi49jb.wav 18391 -0.0 1.0
CUT New length = 10000
1 ./test\1 lasha62hb.wav 22106 0.0 0.9999999
CUT New length = 10000
2 ./test\1 mamuka68hb.wav 18205 -0.0 1.0
CUT New length = 10000
3 ./test\1 me_ @^.wav 30465 -0.0 1.0
CUT New length = 10000
4 ./test\1 saba£##.wav 19134 0.0 0.9999999
CUT New length = 10000
5 ./test\1_ana.wav 5803 -0.0 1.0
PAD New length = 10000
6 ./test\1_giorgi.wav 6827 -0.0 1.0
PAD New length = 10000
7 ./test\1_nika.wav 7510 -0.0 1.0
PAD New length = 10000
8 ./test\1_nina.wav 8875 -0.0 1.0
PAD New length = 10000
9 ./test\1_papa.wav 8363 -0.0 1.0
PAD New length = 10000
10 ./test\1_valera.wav 9217 -0.0 1.0
PAD New length = 10000
11 ./test\1_vato.wav 11265 -0.0 1.0
CUT New length = 10000
12 ./test\1_vatu.wav 7339 0.0 1.0
PAD New length = 10000
13 ./test\2 achi48jj.wav 13189 0.0 0.99999994
CUT New length = 10000
14 ./test\2 lasha72hb.wav 32137 0.0 0.99999994
CUT New length = 10000
15 ./test\2 mamuka38hb.wav 18205 0.0 1.0
CUT New length = 10000
16 ./test\2 me€

In [5]:
import tensorflow
import keras
from keras.utils.np_utils import to_categorical

import numpy as np
import pickle
import os
from glob import glob
import tensorflow as tf


AUDIO_LENGTH = 10000
OUTPUT_DIR = './output'
OUTPUT_TRAIN = './pkls'
OUTPUT_TEST = os.path.join(OUTPUT_DIR, 'test')
n_class = 5
batch_size = 128



def get_my_data_pair(file_list):
    def put_into(currfilename, currx, curry):
        with open(currfilename, 'rb') as f:
            audio_element = pickle.load(f)
            currx.append(audio_element['audio'])
            curry.append(int(audio_element['class_id']))
    x = []
    y = []
    for filename in file_list:
        put_into(filename, x, y)
    return np.array(x), np.array(y)


if __name__ == '__main__':

    x_train, y_train = get_my_data_pair(glob(os.path.join(OUTPUT_TRAIN, '**.pkl')))
    y_train = to_categorical(y_train, num_classes=5)

    print('x train.shape =', x_train.shape)
    print('y train.shape =', y_train.shape)
    
    

Using TensorFlow backend.


x train.shape = (63, 10000, 1)
y train.shape = (63, 5)


In [6]:
 y_train

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0.

In [7]:
from keras.models import load_model


model = load_model('best-trained-model.hdf5')

In [8]:
# loaded_model = pickle.load(open(filename, 'rb'))
result = model.evaluate(x_train, y_train, verbose=1)
print(result)

63/63 [==============================] - 1s 11ms/step
[2.652769550444588, 0.7301587462425232]


In [10]:
import pandas as pd
prediction = model.predict(x_train)
prediction = pd.DataFrame(prediction)
prediction

,0,1,2,3,4
0,0.999512,1.148954e-05,2.345406e-09,4.759675e-04,3.496398e-07
1,0.999994,4.831391e-07,3.716163e-09,5.389706e-06,7.983446e-10
2,0.999996,8.238702e-07,3.219144e-06,9.000295e-08,2.817931e-11
3,1.000000,9.850851e-10,7.085145e-10,4.170606e-07,1.904299e-17
4,0.961459,1.952499e-06,3.106363e-02,7.475703e-03,1.384383e-09
...,...,...,...,...,...
58,0.000096,2.945321e-05,2.176905e-05,3.626414e-04,9.994900e-01
59,0.001082,1.950179e-03,1.736360e-04,7.933011e-03,9.888614e-01
60,0.999915,4.544995e-05,1.637857e-06,3.521904e-05,3.053203e-06
61,0.999837,4.343206e-05,4.426820e-05,5.609370e-06,7.000758e-05


In [14]:
pd.set_option('display.max_rows', prediction.shape[0]+1)

prediction


,0,1,2,3,4
0,9.995122e-01,1.148954e-05,2.345406e-09,4.759675e-04,3.496398e-07
1,9.999942e-01,4.831391e-07,3.716163e-09,5.389706e-06,7.983446e-10
2,9.999958e-01,8.238702e-07,3.219144e-06,9.000295e-08,2.817931e-11
3,9.999996e-01,9.850851e-10,7.085145e-10,4.170606e-07,1.904299e-17
4,9.614587e-01,1.952499e-06,3.106363e-02,7.475703e-03,1.384383e-09
5,6.029252e-05,9.999326e-01,4.664560e-06,2.317439e-06,6.051403e-08
6,2.097588e-02,6.474078e-01,1.654187e-01,8.123965e-02,8.495805e-02
7,3.863654e-03,9.954631e-01,4.924927e-04,8.952594e-05,9.134290e-05
8,3.093385e-13,9.664534e-01,5.462179e-11,3.354659e-02,1.934434e-12
9,6.567475e-08,9.892740e-01,6.853613e-08,7.276147e-07,1.072512e-02


In [123]:
print("validation accuracy : ", result[1])

validation accuracy :  0.8219780325889587


In [82]:
model.fit(x=x_train, y=y_train, epochs=31, batch_size=batch_size, validation_data=(x_test, y_test), callbacks = callbacks_list)


Train on 8034 samples, validate on 910 samples
Epoch 1/31
8034/8034 [==============================] - 401s 50ms/step - loss: 0.2079 - accuracy: 0.9442 - val_loss: 2.1272 - val_accuracy: 0.6099

Epoch 00001: val_accuracy did not improve from 0.82088


C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 2/31
8034/8034 [==============================] - 396s 49ms/step - loss: 0.2324 - accuracy: 0.9344 - val_loss: 1.4961 - val_accuracy: 0.6945

Epoch 00002: val_accuracy did not improve from 0.82088
Epoch 3/31
8034/8034 [==============================] - 396s 49ms/step - loss: 0.2326 - accuracy: 0.9361 - val_loss: 2.2179 - val_accuracy: 0.6132

Epoch 00003: val_accuracy did not improve from 0.82088
Epoch 4/31
8034/8034 [==============================] - 397s 49ms/step - loss: 0.2059 - accuracy: 0.9470 - val_loss: 1.3132 - val_accuracy: 0.7758

Epoch 00004: val_accuracy did not improve from 0.82088
Epoch 5/31
8034/8034 [==============================] - 395s 49ms/step - loss: 0.1878 - accuracy: 0.9515 - val_loss: 0.8038 - val_accuracy: 0.8198

Epoch 00005: val_accuracy did not improve from 0.82088
Epoch 6/31
8034/8034 [==============================] - 396s 49ms/step - loss: 0.1745 - accuracy: 0.9553 - val_loss: 1.2084 - val_accuracy: 0.7571

Epoch 00006: val_accuracy did not improv